In [ ]:
import bpy

pixel_pitch = 0.810547
pixel = 512
size = 0.02
ax_slide_number = 591
ax_slide_distance = 1.25

def load_reference_image(image_path, object_name):
    """
    Load an image as an empty object and assign the specified name.
    """
    try:
        img = bpy.data.images.load(image_path)
    except Exception as e:
        print(f"Failed to load image: {image_path}, Error: {e}")
        return None

    # Add an empty object
    bpy.ops.object.add(type='EMPTY', location=(0, 0, 0))
    empty_obj = bpy.context.object
    empty_obj.empty_display_type = 'IMAGE'
    empty_obj.data = img
    empty_obj.name = object_name

    return empty_obj

class ImageSliderOperator(bpy.types.Operator):
    bl_idname = "object.image_slider_operator"
    bl_label = "Image Slider Operator"

    def execute(self, context):
        path_CT = '/Users/tkimura/Desktop/Blender/TCIA/manifest-1705406030205/JPEG_FULL/'
        axis_type = context.scene.image_slider_axis
        slider_value = context.scene.image_slider_property

        # Common scale
        common_scale = size * pixel_pitch * pixel

        if axis_type == 'AX':  # Axial: Slide along the Y-axis (180-degree rotation along Z-axis)
            ax_slides = ax_slide_number
            image_path = path_CT + 'A/A' + str(slider_value) + '.jpg'
            ax_distance = ax_slide_distance

            # Remove existing image objects
            for obj in bpy.context.scene.objects:
                if obj.name.startswith("iA"):
                    bpy.data.objects.remove(obj, do_unlink=True)

            # Load the image
            image_obj = load_reference_image(image_path, "iA" + str(slider_value))
            if not image_obj:
                return {'CANCELLED'}

            # 180-degree rotation along the Z-axis
            image_obj.rotation_euler = (1.5708, 0, 0)

            # Slide along the Y-axis
            image_obj.location = (0, -(ax_distance / 50) * ((ax_slides + 1) / 2 - slider_value), 0)

            # Adjust the size
            image_obj.scale = (common_scale, common_scale, common_scale)

        elif axis_type == 'COR':  # Coronal: Slide along the Z-axis (180-degree rotation along Y-axis)
            cor_slides = pixel
            image_path = path_CT + 'C/C' + str(slider_value) + '.jpg'
            size_cor = ax_slide_number * ax_slide_distance * size

            # Remove existing image objects
            for obj in bpy.context.scene.objects:
                if obj.name.startswith("iC"):
                    bpy.data.objects.remove(obj, do_unlink=True)

            # Load the image
            image_obj = load_reference_image(image_path, "iC" + str(slider_value))
            if not image_obj:
                return {'CANCELLED'}

            # 180-degree rotation along the Y-axis
            image_obj.rotation_euler = (0, 3.14159, 3.14159)

            # Slide along the Z-axis
            image_obj.location = (0, 0, (pixel_pitch / 50) * (cor_slides / 2 + 0.5 - slider_value))

            # Adjust the size
            image_obj.scale = (size_cor, size_cor, size_cor)

        elif axis_type == 'SAG':  # Sagittal: Same as Coronal (Slide along Z-axis, rotation along Y-axis)
            sag_slides = 512
            image_path = path_CT + 'S/S' + str(slider_value) + '.jpg'
            size_sag = 591 * 1.25 / 50

            # Remove existing image objects
            for obj in bpy.context.scene.objects:
                if obj.name.startswith("iS"):
                    bpy.data.objects.remove(obj, do_unlink=True)

            # Load the image
            image_obj = load_reference_image(image_path, "iS" + str(slider_value))
            if not image_obj:
                return {'CANCELLED'}

            # 180-degree rotation along the Y-axis
            image_obj.rotation_euler = (0, 1.5708, 3.14159)

            # Slide along the Z-axis
            image_obj.location = (-(pixel_pitch / 50) * (sag_slides / 2 + 0.5 - slider_value), 0, 0)

            # Adjust the size
            image_obj.scale = (size_sag, size_sag, size_sag)

        return {'FINISHED'}

class ImageSliderPanel(bpy.types.Panel):
    bl_label = "Image Slider Panel"
    bl_idname = "OBJECT_PT_image_slider"
    bl_space_type = 'VIEW_3D'
    bl_region_type = 'UI'
    bl_category = 'Image Slider'

    def draw(self, context):
        layout = self.layout
        layout.prop(context.scene, 'image_slider_axis', expand=True)  # Axis selection
        layout.prop(context.scene, 'image_slider_property', slider=True)  # Slider

def update_slider(self, context):
    bpy.ops.object.image_slider_operator()

def register():
    bpy.utils.register_class(ImageSliderOperator)
    bpy.utils.register_class(ImageSliderPanel)
    bpy.types.Scene.image_slider_property = bpy.props.IntProperty(name="Slide", min=1, max=591, default=1, update=update_slider)
    bpy.types.Scene.image_slider_axis = bpy.props.EnumProperty(
        name="Axis",
        items=[
            ('AX', 'Axial', 'Axial slices'),
            ('COR', 'Coronal', 'Coronal slices'),
            ('SAG', 'Sagittal', 'Sagittal slices')
        ],
        default='AX'
    )

def unregister():
    bpy.utils.unregister_class(ImageSliderOperator)
    bpy.utils.unregister_class(ImageSliderPanel)
    del bpy.types.Scene.image_slider_property
    del bpy.types.Scene.image_slider_axis

if __name__ == "__main__":
    register()
